In [1]:
from google.colab import userdata
import os

api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

os.environ["OPENAI_API_BASE"] = "https://api.aimlapi.com/v1"

In [2]:
!pip install -q langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.5 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableSequence, RunnableParallel
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

##Setup Streaming Model

In [4]:
# Streaming handler prints output live
streaming_handler = StreamingStdOutCallbackHandler()

model_stream = ChatOpenAI(
    model="gpt-4o-mini",
    streaming=True,  # enable streaming
    callbacks=[streaming_handler]
)

##Prompt + Parser

In [5]:
prompt = ChatPromptTemplate.from_template("Write a short motivational quote about {topic}.")
parser = StrOutputParser()

##Create a RunnableSequence

In [6]:
chain = RunnableSequence(first=prompt, middle=[model_stream], last=parser)

###The output will stream live

In [7]:
result = chain.invoke({"topic": "perseverance"})
print("\nFinal Parsed Result:", result)

"Perseverance is the bridge between dreams and reality; every step forward is a testament to your strength."
Final Parsed Result: "Perseverance is the bridge between dreams and reality; every step forward is a testament to your strength."


##Batching Inputs

In [8]:
inputs = [
    {"topic": "courage"},
    {"topic": "teamwork"},
    {"topic": "innovation"}
]

batch_results = chain.batch(inputs)
print(batch_results)

"Courage is not the absence"Innovation of fear is the, but spark that the strength transforms ideas to face it with into possibilities"Together unwavering, resolve we."; embrace it can and achieve watch the what we future unfold." never could alone—teamwork turns dreams into reality."['"Courage is not the absence of fear, but the strength to face it with unwavering resolve."', '"Together, we can achieve what we never could alone—teamwork turns dreams into reality."', '"Innovation is the spark that transforms ideas into possibilities; embrace it and watch the future unfold."']


##RunnableParallel (Run multiple chains at once)

In [15]:
from langchain.schema.runnable import RunnableParallel, RunnableSequence, RunnableLambda
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# --- Streaming model ---
streaming_handler = StreamingStdOutCallbackHandler()
model_stream = ChatOpenAI(model="gpt-4o-mini", streaming=True, callbacks=[streaming_handler])

# --- Prompt & parser ---
prompt = ChatPromptTemplate.from_template("Write a short motivational quote about {topic}.")
parser = StrOutputParser()

# --- Base sequence ---
seq = RunnableSequence(first=prompt, middle=[model_stream], last=parser)

# --- Wrap each branch to pick its input ---
branch1 = RunnableLambda(lambda x: seq.invoke(x["branch1"]))
branch2 = RunnableLambda(lambda x: seq.invoke(x["branch2"]))

# --- RunnableParallel: dict of branches ---
parallel_chain = RunnableParallel({"branch1": branch1, "branch2": branch2})

# --- Input dict must contain keys for each branch ---
inputs = {
    "branch1": {"topic": "focus"},
    "branch2": {"topic": "happiness"}
}

# --- Invoke parallel chain ---
result_parallel = parallel_chain.invoke(inputs)
print(result_parallel)

"Happiness is not a destination, but a choice we make each day—embrace the little things and let joy be your guide.""Focus is the lens that sharpens your vision; without it, your dreams may blur."{'branch1': '"Focus is the lens that sharpens your vision; without it, your dreams may blur."', 'branch2': '"Happiness is not a destination, but a choice we make each day—embrace the little things and let joy be your guide."'}
